In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

In [3]:
import sentencepiece as spm

vocab = '../sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100
    
    def encode(self, s):
        return self.sp.EncodeAsIds(s)
    
    def decode(self, ids, strip_extraneous=False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [4]:
from tqdm import tqdm
from glob import glob

@registry.register_problem
class Seq2Seq(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 32100
    
    @property
    def is_generate_per_split(self):
        return False
            
    def feature_encoders(self, data_dir):
        encoder = Encoder(sp)
        return {
            "inputs": encoder,
            "targets": encoder
        }

In [5]:
DATA_DIR = os.path.expanduser('t2t-knowledge-graph/data')
TMP_DIR = os.path.expanduser('t2t-knowledge-graph/tmp')

In [6]:
PROBLEM = 'seq2_seq'
t2t_problem = problems.problem(PROBLEM)

In [7]:
import tensorflow as tf
import os

ckpt_path = tf.train.latest_checkpoint('t2t-knowledge-graph/train-base')
ckpt_path

't2t-knowledge-graph/train-base/model.ckpt-325000'

In [8]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

In [9]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.maxlen_decode = tf.reduce_max(self.X_seq_len)
        #self.maxlen_decode = tf.placeholder(tf.int32, None)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        self.features = features
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        self.translate_model = translate_model
        logits, _ = translate_model(features)
        self.logits = logits
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, self.maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, self.maxlen_decode, beam_size=3, 
                top_beams=1, alpha=0.5)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')
        
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32100_512.targets_bottom


INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_32100_512.top


INFO:tensorflow:Transforming body output with symbol_modality_32100_512.top


In [40]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 't2t-knowledge-graph/train-base/model.ckpt-200000')

INFO:tensorflow:Restoring parameters from t2t-knowledge-graph/train-base/model.ckpt-200000


INFO:tensorflow:Restoring parameters from t2t-knowledge-graph/train-base/model.ckpt-200000


In [29]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string)).strip()

In [30]:
# encoded = encoder.encode(cleaning("Ketua negara Malaysia ialah Yang di Pertuan Agong iaitu raja elektif yang terpilih dan diundi dari kalangan sembilan raja negeri Melayu Ketua kerajaannya pula ialah Perdana Menteri.")) + [1]
# f = sess.run([model.nucleus_result], feed_dict = {model.X: [encoded], model.top_p: 0.9})
# encoder.decode(f[0][0].tolist())

In [35]:
encoded = encoder.encode(cleaning("Yang Berhormat Dato Sri Haji Mohammad Najib bin Tun Haji Abdul Razak ialah ahli politik Malaysia dan merupakan bekas Perdana Menteri Malaysia ke-6 yang mana beliau menjawat jawatan dari 3 April 2009 hingga 9 Mei 2018. Beliau juga pernah berkhidmat sebagai bekas Menteri Kewangan dan merupakan Ahli Parlimen Pekan Pahang")) + [1]
f = sess.run([model.fast_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0][0].tolist())

'Honored Dato Sri Haji Mohammad Najib bin Tun Haji Abdul Razak occupation Politician, educated at Pekan Pahang, position held Member of the 2018 Parliament of Malaysia, member of 6th Prime Minister of Malaysia, position held Member of the 2018 Parliament of Malaysia, honorific prefix The Imam, position held Member of the 2018 Parliament of Malaysia.'

In [41]:
encoded = encoder.encode(cleaning("Pahang ialah negeri yang ketiga terbesar di Malaysia Terletak di lembangan Sungai Pahang yang amat luas negeri Pahang bersempadan dengan Kelantan di utara Perak Selangor serta Negeri Sembilan di barat Johor di selatan dan Terengganu dan Laut China Selatan di timur.")) + [1]
f = sess.run([model.fast_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0][0].tolist())

'Pahang , Malaysia located in or next to body of water Pahang, shares border with Kelantan, located in or next to body of water Pahang, shares border with Perak Selangor, located in or next to body of water North, shares border with Negeri Sembilan, located in or next to body of water Terengganu.'

In [16]:
files = tf.gfile.Glob('../t2t-ms-en/data/*')
len(files)

In [17]:
# batch_size = 1
# data_fields = {
#     'inputs': tf.VarLenFeature(tf.int64),
#     'targets': tf.VarLenFeature(tf.int64),
# }
# data_len = {
#     'inputs': 300,
#     'targets': 300,
# }

# def parse(serialized_example):

#     features = tf.parse_single_example(
#         serialized_example, features = data_fields
#     )
#     for k in features.keys():
#         features[k] = features[k].values
#         features[k] = tf.pad(
#             features[k], [[0, data_len[k] - tf.shape(features[k])[0]]]
#         )
#         features[k].set_shape((data_len[k]))

#     return features

# def _decode_record(example, name_to_features):
#     """Decodes a record to a TensorFlow example."""

#     # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
#     # So cast all int64 to int32.
#     for name in list(example.keys()):
#         t = example[name]
#         if t.dtype == tf.int64:
#             t = tf.to_int32(t)
#         example[name] = t

#     return example

# d = tf.data.TFRecordDataset(files)
# d = d.map(parse, num_parallel_calls = 32)
# d = d.apply(
#     tf.contrib.data.map_and_batch(
#         lambda record: _decode_record(record, data_fields),
#         batch_size = batch_size,
#         num_parallel_batches = 4,
#         drop_remainder = True,
#     )
# )
# d = d.make_one_shot_iterator().get_next()
# d

In [18]:
# r_ = sess.run(d)
# encoder.decode(r_['inputs'][0].tolist()), encoder.decode(r_['targets'][0].tolist())

In [19]:
# encoder.decode(r_['inputs'][2].tolist())

In [20]:
# encoder.decode(r_['targets'][2].tolist())

In [42]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'transformer-base/model.ckpt')

'transformer-base/model.ckpt'

In [43]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)

In [44]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [45]:
freeze_graph('transformer-base', strings)

INFO:tensorflow:Restoring parameters from transformer-base/model.ckpt


INFO:tensorflow:Restoring parameters from transformer-base/model.ckpt


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Converted 201 variables to const ops.


INFO:tensorflow:Converted 201 variables to const ops.


11004 ops in the final graph.


In [46]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [47]:
import tensorflow_text
import tf_sentencepiece

In [48]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'transformer-base/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

print(pb)

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['greedy', 'beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

transformer-base/frozen_model.pb
